<a href="https://colab.research.google.com/github/Nancy240/Multi-Modal-RAG-System/blob/main/Advanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -Uq "unstructured[all-docs]" pillow lxml
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-groq
%pip install -Uq transformers accelerate torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 32.1 MB/s eta 0:00:00
   ━

In [ ]:
!apt-get install -y -q poppler-utils

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (302 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import os
import uuid
import gc
import torch
import base64
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from unstructured.partition.pdf import partition_pdf
import gradio as gr

In [ ]:
HF_TOKEN = userdata.get("HF_TOKEN")
os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [ ]:
file_path = "/content/EJ1172284.pdf"

In [ ]:
chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,
    strategy="hi_res",
    extract_image_block_types=["Image"],
    extract_image_block_to_payload=True,
    chunking_strategy="by_title",
    max_characters=10000,
    combine_text_under_n_chars=2000,
    new_after_n_chars=6000,
)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [ ]:
tables, texts, images = [], [], []

for chunk in chunks:
    if "Table" in str(type(chunk)):
        tables.append(chunk)
    elif "CompositeElement" in str(type(chunk)):
        texts.append(chunk)
        for el in chunk.metadata.orig_elements:
            if "Image" in str(type(el)):
                images.append(el.metadata.image_base64)

In [ ]:
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additional comment.
Table or text chunk: {element}
"""
prompt = ChatPromptTemplate.from_template(prompt_text)
model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})
tables_html = [table.metadata.text_as_html for table in tables]
table_summaries = summarize_chain.batch(tables_html, {"max_concurrency": 3})
image_summaries = ["This is an image. (Base64)" for _ in images]

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=embedding_model)
docstore = InMemoryStore()
id_key = "doc_id"
retriever = MultiVectorRetriever(vectorstore=vectorstore, docstore=docstore, id_key=id_key)

/tmp/ipython-input-11-1538273978.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-11-1538273978.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=embedding_model)


In [ ]:
import uuid
from langchain.schema import Document

# TEXT DOCUMENTS
print(f"Text summaries: {len(text_summaries)}")
text_docs = [
    (str(uuid.uuid4()), summary, texts[i])
    for i, summary in enumerate(text_summaries)
    if summary.strip()
]
summary_docs = [
    Document(page_content=summary, metadata={id_key: doc_id})
    for doc_id, summary, _ in text_docs
]
print(f"Text documents to add: {len(summary_docs)}")
if summary_docs:
    retriever.vectorstore.add_documents(summary_docs)
    docstore.mset([
        (doc_id, Document(page_content=str(orig_text), metadata={"type": "text"}))
        for doc_id, _, orig_text in text_docs
    ])

# TABLE DOCUMENTS
print(f"Table summaries: {len(table_summaries)}")
table_docs_data = [
    (str(uuid.uuid4()), summary, tables[i])
    for i, summary in enumerate(table_summaries)
    if summary.strip()
]
table_docs = [
    Document(page_content=summary, metadata={id_key: doc_id})
    for doc_id, summary, _ in table_docs_data
]
print(f"Table documents to add: {len(table_docs)}")
if table_docs:
    retriever.vectorstore.add_documents(table_docs)
    docstore.mset([
        (doc_id, Document(page_content=str(orig_table), metadata={"type": "table"}))
        for doc_id, _, orig_table in table_docs_data
    ])

# IMAGE DOCUMENTS
print(f"Image summaries: {len(image_summaries)}")
img_docs_data = [
    (str(uuid.uuid4()), summary, images[i])
    for i, summary in enumerate(image_summaries)
    if summary.strip()
]
img_docs = [
    Document(page_content=summary, metadata={id_key: doc_id})
    for doc_id, summary, _ in img_docs_data
]
print(f"Image documents to add: {len(img_docs)}")
if img_docs:
    retriever.vectorstore.add_documents(img_docs)
    docstore.mset([
        (doc_id, Document(page_content=str(orig_image), metadata={"type": "image"}))
        for doc_id, _, orig_image in img_docs_data
    ])


Text summaries: 8
Text documents to add: 8
Table summaries: 0
Table documents to add: 0
Image summaries: 11
Image documents to add: 11


In [ ]:
model_path = "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.2)
llm = HuggingFacePipeline(pipeline=pipe)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-13-1100381642.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
# # Clear RAM Helper Function
# def unload_model():
#     global model, pipe
#     del model
#     del pipe
#     gc.collect()
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()

In [ ]:
rag_prompt_template = """
You are a helpful assistant. Use the context below to answer the question.
If you don't know the answer, say you don't know because that kind of information is not provided.

Context:
{context}

Question:
{question}

Helpful Answer:
"""
rag_prompt = PromptTemplate(template=rag_prompt_template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True, chain_type_kwargs={"prompt": rag_prompt})


In [ ]:
def ask_question(question):
    result = qa_chain.invoke({"query": question})
    print("🔍 Question:", question)
    print("\n📘 Answer:\n", result["result"])

    print("\n📚 Source Documents:")
    for i, doc in enumerate(result["source_documents"], 1):
        print(f"\n--- Source {i} ---\n{doc.page_content[:1000]}")


In [ ]:
question = input("Enter your question: ")
ask_question(question)

Enter your question: Summary of the pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔍 Question: Summary of the pdf

📘 Answer:
 
You are a helpful assistant. Use the context below to answer the question.
If you don't know the answer, say you don't know because that kind of information is not provided.

Context:
References

Benson, P. (2001). Teaching and researching autonomy in language learning. Harlow: Pearson Education.

Benson, P. (2011). What’s new in autonomy? The Language Teacher, 35(4), 15-18.

Benson, P. & Chik, A. (2010). New literacies and autonomy in foreign language learning. In M. J. Luzón, M. N. Ruiz-Madrid & M. L. Villanueva (Eds.), Digital genres, new literacies, and autonomy in language learning (pp. 63-80). Newcastle-upon-Tyne: Cambridge Scholars Publishing.

Byrne, J. & Diem, R. (2014). Profiling mobile English language learners. The JALT CALL Journal I, 10 (1), 3-19.

Cakir, I. (2015). Opinions and attitudes of prospective teachers for the use of mobile phones in foreign language learning. Contemporary Educational Technology, 6(3), 239- 255.

Cavus